In [47]:
from time import sleep

sleep(54000)

In [48]:
import os
import keras

base_dir = '/home/wkeeley/Desktop/Brain-Hemorrhaging'
image_dir = os.path.join(base_dir, 'Images')
train_dir = os.path.join(image_dir, 'Train_Images')
val_dir = os.path.join(image_dir, 'Validation_Images')

dcm_dir = os.path.join(base_dir, 'stage_1_train_images')
dcm_test_dir = os.path.join(base_dir, 'stage_1_test_images')

sub_types = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']

sub_train_dirs = []
sub_val_dirs = []

for i in sub_types:
    sub_train_dirs.append(os.path.join(train_dir, i))
    sub_val_dirs.append(os.path.join(val_dir, i))

In [49]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15995580734329913769
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1810341489240227101
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8449854057911645205
physical_device_desc: "device: XLA_CPU device"
]


In [60]:

from keras import layers
from keras import models
from keras import Input

input_tensor = Input(shape=(256, 256, 3))

branch_a = layers.Conv2D(256, 1, strides=2, activation='relu')(input_tensor)
branch_a = layers.Conv2D(128, 4, padding='same', strides=2, activation='relu')(branch_a)
print(branch_a.shape)

branch_b = layers.SeparableConv2D(256, 1, activation='relu')(input_tensor)
branch_b = layers.Dropout(0.2)(branch_b)
branch_b = layers.MaxPooling2D(strides=2)(branch_b)
branch_b = layers.SeparableConv2D(128, 2, padding='same', activation='relu')(branch_b)
print(branch_b.shape)

'''branch_b = layers.SeparableConv2D(256, 1, activation='relu')(input_tensor)
branch_b = layers.SeparableConv2D(256, 4, padding='same', activation='relu')(branch_b)
branch_b = layers.SpatialDropout2D(0.2)(branch_b)
branch_b = layers.AveragePooling2D(2, strides=2)(branch_b)
branch_b = layers.SeparableConv2D(256, 2, padding='same', activation='relu')(branch_b)
branch_b = layers.SeparableConv2D(256, 2, padding='same', activation='relu')(branch_b)
print(branch_b.shape)

branch_d = layers.SeparableConv2D(256, 1, activation='relu')(input_tensor)
branch_d = layers.SeparableConv2D(256, 1, activation='relu')(branch_d)
branch_d = layers.SeparableConv2D(256, 1, activation='relu')(branch_d)
branch_d = layers.SeparableConv2D(256, 1, activation='relu', strides=2)(branch_d)
print(branch_d.shape)'''


#out = layers.concatenate([branch_a, branch_b], axis=-1)
out = layers.Flatten()(branch_b)
out = layers.Dense(1, activation='sigmoid')(out)
print(out.shape)



(?, 64, 64, 128)
(?, 128, 128, 128)
(?, 1)


In [61]:

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [62]:

def callbacks(category):
    log_dir = os.path.join(base_dir, category + '_logs')
    try:
        os.mkdir(log_dir)
    except FileExistsError:
        pass
    
    callbacks = [
    keras.callbacks.EarlyStopping(monitor='acc', patience=2),
    keras.callbacks.ModelCheckpoint(category + '.h5', 
                                    monitor='val_loss', 
                                    save_best_only=True),
    keras.callbacks.TensorBoard(log_dir=log_dir)
    ]
    return callbacks
    
    

In [64]:
#epidural
epidural_data = train_datagen.flow_from_directory(sub_train_dirs[0],
                                           target_size=(256, 256),
                                           batch_size=128,
                                           class_mode='binary')

epidural_val_data = val_datagen.flow_from_directory(sub_val_dirs[0],
                                                       target_size=(256, 256),
                                                       batch_size=128,
                                                       class_mode='binary')
epidural_callbacks = callbacks('epidural')

Found 606833 images belonging to 2 classes.
Found 67425 images belonging to 2 classes.


In [65]:
epidural_data.batch_size

128

In [66]:
epidural_model = keras.Model(inputs=input_tensor, output=out)

epidural_model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-3),
              loss='binary_crossentropy',
              metrics=['acc'])
epidural_model.summary()

/home/wkeeley/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 256, 256, 3)       0         
_________________________________________________________________
separable_conv2d_35 (Separab (None, 256, 256, 256)     1027      
_________________________________________________________________
dropout_14 (Dropout)         (None, 256, 256, 256)     0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 128, 128, 256)     0         
_________________________________________________________________
separable_conv2d_36 (Separab (None, 128, 128, 128)     33920     
_________________________________________________________________
flatten_10 (Flatten)         (None, 2097152)           0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 2097

In [67]:
epidural_data.samples//epidural_data.batch_size

4740

In [ ]:
history = epidural_model.fit_generator(epidural_data,
                           steps_per_epoch=epidural_data.samples//epidural_data.batch_size,
                           epochs=64,
                           callbacks=epidural_callbacks,
                           validation_data=epidural_val_data,
                           validation_steps=epidural_val_data.samples//epidural_val_data.batch_size,
                           verbose=1,
                           use_multiprocessing=True,
                           workers=8)

Epoch 1/64


In [ ]:
intraparenchymal_data = train_datagen.flow_from_dir(sub_train_dirs[1],
                                           target_size=(256, 156),
                                           batch_size=128,
                                           class_mode='binary')

intraparenchymal_val_data = test_datagen.flow_from_directory(sub_val_dirs[1],
                                                       target_size=(256, 256),
                                                       batch_size=128,
                                                       class_mode='binary')

In [ ]:
intraparenchymal_callbacks = callbacks('intraparenchymal')
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
history = out.fit_generator(intraparenchymal_data,
                           steps_per_epoch=100,
                           epochs=30,
                           callbacks=intraparenchymal_callbacks,
                           validation_data=intraparenchymal_val_data,
                           validation_steps=50)